In [1]:
from sklearn.datasets import load_iris
import numpy as np

In [17]:
#使用鸢尾花数据集,该数据集用于无监督学习的KMeans中，y无需使用
X,y = load_iris(return_X_y=True)

In [79]:
def randomCents(dataSet,k):
    '''
    随机生成k个聚类中心
    
    dataSet:无标签的数据集
    k:要形成簇的数量
    '''
    m,n = dataSet.shape
    
    max_value = max(dataSet.max(axis=0))#数据集中的最大值
    min_value = min(dataSet.min(axis=0))#数据集中的最小值
    
    #k个簇就会有k个中心,所以中心的维度为k行,n列
    k_cents = np.random.randint(min_value,max_value+2,(k,n))+np.random.rand(k,n)
    
    return k_cents

def euclid_distance(dataSet,cent):
    '''
    使用欧几里得距离来计算每一个样本到质心的距离
    dataSet:需要求距离的数据集
    cent:一个中心
    '''
    k = cent.shape[0]
    return np.sum((dataSet-cent)**2,axis=1) #全部数据集到cent点的距离
    
#KMeans方法进行聚类
def KMeans(dataSet,k=3,measure_method=euclid_distance,min_iterator=1000):
    '''
    dataSet:无标签的数据集
    k:要形成簇的数量
    measure_method:距离测量的方法，默认使用欧氏距离
    min_iterator:聚类操作最小的迭代次数
    '''
    #随机生成k个中心
    k_cents = randomCents(dataSet,k)
    #结束迭代的标识。当类簇中心不再发生变化时，end=True停止迭代
    end = False
    iterator = 1
    while not end or iterator<=min_iterator:
        #在某个簇中的数据集的角标,数据格式{"cluster1":[1,3,5,7]}表示cluster1中有dataSet[1],dataSet[3]
        cluster_map = {}
        #k_distances:(dataSet.row,k),k列表示数据到第k个簇的距离
        k_distances = None
        #计算数据集到每个簇的距离
        for i in range(k):
            distances = euclid_distance(dataSet,k_cents[i])
            distances = distances.reshape(distances.shape[0],1)
            if k_distances is None:
                k_distances = distances
            else:
                k_distances = np.concatenate((k_distances,distances),axis=1)
        #根据距离判断一行数据所属于哪一个簇
        #data_clusters存储的就是数据所属的类别
        data_clusters = np.argsort(k_distances,axis=1)[:,0].reshape(k_distances.shape[0],1)#按行升序排序后再选择第一列即为最小距离所在的角标
        
        t = 0 #记录是否k个簇中心都不变
        for i in range(k):
            #dataset_index_in_cluster属于i类的行号
            dataset_index_in_cluster = np.array(np.where(data_clusters==i))[0]
            #簇中没有元素，中心不变
            if len(dataset_index_in_cluster) == 0:
                cent = k_cents[i]
            else:
                cent = dataSet[dataset_index_in_cluster].mean(axis=0)
            #如果cent和k_cents[i]中的每一个元素都相等
            if (cent == k_cents[i]).astype("int").sum(axis=0) == cent.shape[0]:
                t += 1
            else:
                k_cents[i] = cent
            
            #保存簇中的元素
            cluster_map["cluster"+str(i+1)] = dataset_index_in_cluster   
            
        if t == k:
            end = True
        
        iterator+=1
            
    for i in range(k):
        print("第%d个聚类中心信息："%(i+1))
        print(k_cents[i])
        print(cluster_map["cluster"+str(i+1)])


In [80]:
KMeans(X,3)

第1个聚类中心信息：
[ 5.006  3.418  1.464  0.244]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49]
第2个聚类中心信息：
[ 5.9016129   2.7483871   4.39354839  1.43387097]
[ 50  51  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  74  75  76  78  79  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  94  95  96  97  98  99 101 106 113 114 119 121
 123 126 127 133 138 142 146 149]
第3个聚类中心信息：
[ 6.85        3.07368421  5.74210526  2.07105263]
[ 52  77 100 102 103 104 105 107 108 109 110 111 112 115 116 117 118 120
 122 124 125 128 129 130 131 132 134 135 136 137 139 140 141 143 144 145
 147 148]


In [81]:
cents = randomCents(X,3)
distances = euclid_distance(X,cents[0])
distances = distances.reshape(distances.shape[0],1)
a=np.concatenate((distances,distances),axis=1)